In [1]:
import pickle
import pandas as pd

from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
from typing import Callable
from src.config import Config
from src.dataset import Interactions
from src.metrics import PrecisionAtK, RecallAtK, FOneScoreAtK
from src.optimizer import OptunaMaximizer

In [2]:
with Config.PREPROCESSED_INTERACTIONS_PATH.open("rb") as file_object:
    interactions = Interactions(pickle.load(file_object))

In [3]:
interactions.train = interactions.train[["user_id", "item_id", "interest_score"]]
interactions.test = interactions.test[["user_id", "item_id", "interest_score"]]
interactions.valid = interactions.valid[["user_id", "item_id", "interest_score"]]

In [4]:
interactions.train

,user_id,item_id,interest_score
0,126706,14433,0.400
1,127290,140952,0.290
2,66991,198453,0.445
3,46791,83486,0.615
4,79313,188770,0.940
...,...,...,...
1532993,153908,98585,0.220
1532994,154008,251969,0.020
1532995,154892,298192,0.840
1532996,156948,38118,0.890


In [5]:
interactions.test

,user_id,item_id,interest_score
1517914,38753,135245,0.000
1517915,101642,319500,0.835
1517916,13548,251184,0.000
1517917,130425,193445,0.490
1517918,93986,80733,0.235
...,...,...,...
1530838,141930,219928,0.450
1530839,53358,42887,0.290
1530840,151170,284652,0.135
1530841,141293,273421,0.240


In [6]:
interactions.valid

,user_id,item_id,interest_score
1503047,22032,287219,0.275
1503048,84214,121609,1.000
1503049,28992,11482,0.110
1503050,23345,281921,1.000
1503051,49466,2880,0.010
...,...,...,...
1517909,138587,291806,0.000
1517910,158991,99669,0.815
1517911,77232,142149,0.020
1517912,17843,174535,0.060


In [7]:
csr_train = csr_matrix(
    (interactions.train["interest_score"], 
    (interactions.train["user_id"], interactions.train["item_id"]))
)
csr_train

<159613x321752 sparse matrix of type '<class 'numpy.float32'>'
	with 1505202 stored elements in Compressed Sparse Row format>

In [8]:
class Objective:
    def __init__(self, train: csr_matrix, valid: pd.DataFrame, metric: Callable):
        self.train = train
        self.valid = valid
        self.metric = metric

    def __call__(self, trial) -> float:
        search_space = {
            "factors": trial.suggest_int("factors", 4, 256),
            "regularization": trial.suggest_float("regularization", 1e-8, 0.1),
            "iterations": trial.suggest_int("iterations", 8, 64)
        }
        model = AlternatingLeastSquares(**search_space)
        model.fit(self.train)
        predictions = pd.DataFrame(self.valid["user_id"].drop_duplicates())
        predictions["item_id"] = predictions["user_id"].apply(
            lambda user_id: 
                model.recommend(
                    user_id, 
                    self.train[user_id],
                    N=Config.K,
                    filter_already_liked_items=True
                )[0]
        )
        predictions = predictions.explode("item_id")
        return self.valid["user_id"].drop_duplicates().apply(
            lambda user_id: 
                self.metric.calculate(
                    predictions[predictions["user_id"] == user_id]["item_id"],
                    self.valid[self.valid["user_id"] == user_id]["item_id"]
                )
        ).mean()

In [9]:
recall = RecallAtK(Config.K)
precision = PrecisionAtK(Config.K)
fscore = FOneScoreAtK(Config.K)

In [10]:
fscore_objective = Objective(csr_train, interactions.valid, fscore)
fscore_optimizer = OptunaMaximizer(fscore_objective, "ALS fscore maximize")
fscore_best_trial = fscore_optimizer.optimize(Config.ITERATIONS, Config.STOP_ITERATION)

[I 2023-02-25 16:46:10,183] A new study created in memory with name: ALS fscore maximize
/home/aveitsme/miniconda3/envs/core/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

[I 2023-02-25 16:46:31,008] Trial 0 finished with value: 0.0037711276860178285 and parameters: {'factors': 224, 'regularization': 0.04394813418473055, 'iterations': 20}. Best is trial 0 with value: 0.0037711276860178285.


  0%|          | 0/22 [00:00<?, ?it/s]

...

[I 2023-02-25 17:08:32,977] Trial 59 finished with value: 0.0037243870342860224 and parameters: {'factors': 228, 'regularization': 0.020301595454325024, 'iterations': 17}. Best is trial 41 with value: 0.004258197826611783.


  0%|          | 0/23 [00:00<?, ?it/s]

[I 2023-02-25 17:08:55,937] Trial 60 finished with value: 0.0037022229596914338 and parameters: {'factors': 244, 'regularization': 0.0037327770716060645, 'iterations': 23}. Best is trial 41 with value: 0.004258197826611783.


  0%|          | 0/8 [00:00<?, ?it/s]

[I 2023-02-25 17:09:16,726] Trial 61 finished with value: 0.004033995717416129 and parameters: {'factors': 242, 'regularization': 0.014811080425575985, 'iterations': 8}. Best is trial 41 with value: 0.004258197826611783.
EarlyStopping Exceeded: No new best scores on iters 20


In [11]:
precision_objective = Objective(csr_train, interactions.valid, precision)
precision_optimizer = OptunaMaximizer(precision_objective, "ALS precision maximize")
precision_best_trial = precision_optimizer.optimize(Config.ITERATIONS, Config.STOP_ITERATION)

[I 2023-02-25 17:09:16,824] A new study created in memory with name: ALS precision maximize
/home/aveitsme/miniconda3/envs/core/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

[I 2023-02-25 17:09:34,370] Trial 0 finished with value: 0.0020943414352963814 and parameters: {'factors': 143, 'regularization': 0.01044289520634905, 'iterations': 37}. Best is trial 0 with value: 0.0020943414352963814.


  0%|          | 0/55 [00:00<?, ?it/s]

...

[I 2023-02-25 17:21:29,887] Trial 33 finished with value: 0.0024498030550485226 and parameters: {'factors': 255, 'regularization': 0.051369258325612725, 'iterations': 30}. Best is trial 15 with value: 0.002526659621481417.


  0%|          | 0/19 [00:00<?, ?it/s]

[I 2023-02-25 17:21:50,342] Trial 34 finished with value: 0.0024786242674608577 and parameters: {'factors': 211, 'regularization': 0.05940959960077833, 'iterations': 19}. Best is trial 15 with value: 0.002526659621481417.


  0%|          | 0/19 [00:00<?, ?it/s]

[I 2023-02-25 17:22:10,366] Trial 35 finished with value: 0.0024498030550485217 and parameters: {'factors': 214, 'regularization': 0.059080796121788764, 'iterations': 19}. Best is trial 15 with value: 0.002526659621481417.
EarlyStopping Exceeded: No new best scores on iters 20


In [12]:
recall_objective = Objective(csr_train, interactions.valid, recall)
recall_optimizer = OptunaMaximizer(recall_objective, "ALS recall maximize")
recall_best_trial = recall_optimizer.optimize(Config.ITERATIONS, Config.STOP_ITERATION)

[I 2023-02-25 17:22:10,438] A new study created in memory with name: ALS recall maximize
/home/aveitsme/miniconda3/envs/core/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

[I 2023-02-25 17:22:31,996] Trial 0 finished with value: 0.00873427491318259 and parameters: {'factors': 133, 'regularization': 0.06830735063505491, 'iterations': 59}. Best is trial 0 with value: 0.00873427491318259.


  0%|          | 0/15 [00:00<?, ?it/s]

...

[I 2023-02-25 17:46:31,555] Trial 71 finished with value: 0.012391246764145216 and parameters: {'factors': 240, 'regularization': 0.07823096783354283, 'iterations': 8}. Best is trial 53 with value: 0.013415980248998914.


  0%|          | 0/13 [00:00<?, ?it/s]

[I 2023-02-25 17:46:52,047] Trial 72 finished with value: 0.011925906871386746 and parameters: {'factors': 250, 'regularization': 0.07105968486608927, 'iterations': 13}. Best is trial 53 with value: 0.013415980248998914.


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2023-02-25 17:47:11,644] Trial 73 finished with value: 0.012054916107899103 and parameters: {'factors': 229, 'regularization': 0.07605414774879986, 'iterations': 10}. Best is trial 53 with value: 0.013415980248998914.
EarlyStopping Exceeded: No new best scores on iters 20


In [13]:
precision_model = AlternatingLeastSquares(**precision_best_trial.params)
precision_model.fit(csr_train)

  0%|          | 0/63 [00:00<?, ?it/s]

In [14]:
recall_model = AlternatingLeastSquares(**recall_best_trial.params)
recall_model.fit(csr_train)

  0%|          | 0/8 [00:00<?, ?it/s]

In [15]:
fscore_model = AlternatingLeastSquares(**fscore_best_trial.params)
fscore_model.fit(csr_train)

  0%|          | 0/8 [00:00<?, ?it/s]

In [16]:
models = {
    "fscore_model": fscore_model,
    "recall_model": recall_model,
    "precision_model": precision_model
}

In [17]:
metrics = {
    "precision": precision,
    "recall": recall
}

In [18]:
for model_name, model in models.items():
    pred = pd.DataFrame(interactions.test["user_id"].drop_duplicates())
    print(f"Model '{model_name}':")
    pred["item_id"] = pred["user_id"].apply(
        lambda user_id: 
            model.recommend(
                user_id, 
                csr_train[user_id],
                N=Config.K,
                filter_already_liked_items=True
            )[0]
    )
    pred = pred.explode("item_id")
    for metric_name, metric in metrics.items():
        metric_val = interactions.test["user_id"].drop_duplicates().apply(
            lambda user_id: 
                metric.calculate(
                    pred[pred["user_id"] == user_id]["item_id"],
                    interactions.test[interactions.test["user_id"] == user_id]["item_id"]
                )
        ).mean()
        print(f"\tMetric {metric_name}@{Config.K}: {metric_val}")

Model 'fscore_model':
	Metric precision@10: 0.0018104817363684472
	Metric recall@10: 0.0095555761807747
Model 'recall_model':
	Metric precision@10: 0.0019798835362625734
	Metric recall@10: 0.010332467074129329
Model 'precision_model':
	Metric precision@10: 0.0015669666490206434
	Metric recall@10: 0.008093229046114172


In [20]:
with Config.IMPLICIT_ALS_PATH.open("wb") as file_object:
    pickle.dump(recall_model, file_object)